<a href="https://colab.research.google.com/github/rahiakela/computer-vision-research-and-practice/blob/main/hands-on-computer-vision-with-detectron2/04-custom-object-detection/01_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Processing

The dataset used is the brain tumor object detection dataset available from [Kaggle](https://www.kaggle.com/datasets/davidbroberts/brain-tumor-object-detectiondatasets).

This dataset is chosen because medical image processing is a critical subfield in computer vision. At the same
time, the task is challenging, and the number of images is appropriate for demonstration purposes.



##Setup

In [ ]:
!python -m pip install "git+https://github.com/facebookresearch/detectron2.git"
!sudo apt-get install tree
!pip install -q pylabel

In [2]:
import detectron2
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer

import os
from glob import glob
import shutil
from tqdm import tqdm
import yaml
from yaml.loader import SafeLoader
from pylabel import importer

import torch
import cv2

import numpy as np
import matplotlib.pyplot as plt

# Suppress some user warnings
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
print(detectron2.__version__)

0.6


In [ ]:
!wget https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/blob/main/datasets/braintumors.zip?raw=true -O braintumors.zip
!unzip braintumors.zip -d braintumors

##Dataset

In [5]:
data_folder = "braintumors"
data_folder_yolo = data_folder + "_yolo"
data_folder_coco = data_folder + "_coco"

folders = os.listdir("braintumors")
print(folders)

['sagittal_t1wce_2_class', 'coronal_t1wce_2_class', 'axial_t1wce_2_class']


In [6]:
!tree braintumors/ -d

braintumors/
├── axial_t1wce_2_class
│   ├── images
│   │   ├── test
│   │   └── train
│   └── labels
│       ├── test
│       └── train
├── coronal_t1wce_2_class
│   ├── images
│   │   ├── test
│   │   └── train
│   └── labels
│       ├── test
│       └── train
└── sagittal_t1wce_2_class
    ├── images
    │   ├── test
    │   └── train
    └── labels
        ├── test
        └── train

21 directories


In [7]:
# let's count the number of images and labels from a dataset with the YOLO annotation format
def count_yolo_data(folder):
  for images_labels in ["images", "labels"]:
    for train_test in ["train", "test"]:
      file_ext = "*.jpg" if images_labels == "images" else "*.txt"
      p = os.path.join(folder, images_labels, train_test, file_ext)
      files = glob(p)
      print(train_test, images_labels, len(files))

In [8]:
for folder in folders:
  print("-"*8 + folder + "-"*8)
  folder = os.path.join(data_folder, folder)
  count_yolo_data(folder)

--------sagittal_t1wce_2_class--------
train images 264
test images 70
train labels 264
test labels 70
--------coronal_t1wce_2_class--------
train images 319
test images 78
train labels 318
test labels 78
--------axial_t1wce_2_class--------
train images 310
test images 75
train labels 296
test labels 75


In [11]:
# due to the small number of images, it might be more reasonable to combine images from these folders into one folder
def copy_yolo_files(from_folder, to_folder, images_labels, train_set):
  from_path = os.path.join(from_folder, images_labels, train_set)
  to_path = os.path.join(to_folder, images_labels, train_set)
  os.makedirs(to_path, exist_ok=True)

  # get files
  file_ext = "*.jpg" if images_labels == "images" else "*.txt"
  files = glob(os.path.join(from_path, file_ext))
  # move files
  for file in tqdm(files):
    shutil.copy(file, to_path)

In [12]:
# now, combines images from three folders into one
for from_folder in folders:
  from_folder = os.path.join(data_folder, from_folder)
  to_folder = data_folder_yolo
  for images_labels in ["images", "labels"]:
    for train_set in ["train", "test"]:
      copy_yolo_files(from_folder, to_folder, images_labels, train_set)

100%|██████████| 75/75 [00:00<00:00, 5934.10it/s]


In [13]:
# Now the folder should look like
!tree -d {data_folder_yolo}

braintumors_yolo
├── images
│   ├── test
│   └── train
└── labels
    ├── test
    └── train

6 directories


In [14]:
# do the counting just to make sure
count_yolo_data(data_folder_yolo)

train images 893
test images 223
train labels 878
test labels 223


In [15]:
# let's read the yaml file and write the class file
with open(os.path.join(data_folder, folders[0], folders[0] + ".yaml")) as f:
  classes = yaml.load(f, Loader=SafeLoader)["names"]
# Write classes.txt
with open(os.path.join(data_folder_yolo, "classes.txt"), "w") as f:
  f.write("\n".join(classes))

##Data format conversion